In this project you will use and modify the module you developed previously (File Encryption). You'll also use the [OS package](https://docs.python.org/3/library/os.html) as well as [JSON](https://docs.python.org/3/library/json.html) package.

Step 1:

Modify your File Encryption to include the policy of Encrypt-then-[MAC](https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/) for every encryption.

(C, IV, tag)= **MyencryptMAC**(message, EncKey, HMACKey)

(C, IV, tag, Enckey, HMACKey, ext)= **MyfileEncryptMAC** (filepath)

(RSACipher, C, IV, tag, ext)= **MyRSAEncrypt**(filepath, RSA_Publickey_filepath)

Where in the above, RSACipher includes RSA encryption of m=EncKey+ HMACKey (concatenated). Make sure you implement the reverse of the above (MACVerification-then-Decrypt). You can use SHA256 in your [HMAC](https://cryptography.io/en/latest/hazmat/primitives/mac/hmac/).

Save this class as FileEncryptMAC.

Step 2:

Next, you will a script that looks for a pair of RSA Public and private key (using a CONSTANT file path; PEM format). If the files do not exist (use OS package) then generate the RSA public and private key (2048 bits length) using the same constant file path.

Step 3:

You can use the OS package to retrieve the current working directory. Then you can get a list of all files in this directory. For each file, encrypt them using **MyRSAEncrypt** from your new FileEncryptMAC module. Do this in a loop for all files (make sure you do not encrypt the RSA Private Key file). For every file that is encrypted, store the encrypted file as a JSON file. The attributes you have for each file are 'RSACipher', 'C', 'IV', 'tag' and 'ext'. The values are from MyRSAEncrypt method. Once the JSON fire is written (use json.dump() with file.write() methods) into a JSON file then you can remove the plaintext file (use os.remove() method).

Note: For now, you can skip encrypting files within directories in the working directories (i.e., recursive execution).

Note: DO NOT test your script on any valuable file. It will be your responsibility if you lose any important data to you.

Step 4:

Using [Pyinstaller](http://www.pyinstaller.org/) or [Py2exe](http://www.py2exe.org/) create an executable file from your step 3.

Do NOT run the executable file on important folders. Only test on a designated python working directory. You are responsible if you lose any important file.

## (C, IV, tag)= MyencryptMAC(message, EncKey, HMACKey)

In [91]:
import os
import base64
import cryptography
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding, serialization, hashes, asymmetric, hashes, hmac
from cryptography.hazmat.primitives.asymmetric import rsa

# encryption method AES-CBC-256, Encrypt-then-MAC(SHA256)
def MyencryptMAC(message, EncKey, HMACKey):
    
    #step 1: Encrypt
    #key length check
    if len(EncKey)<32:
        return "Error: This place is full of land mines, dragons, and dinosaurs with laser guns. Increase you key length to upgrade your armor."
    
    try:
        message = message.encode()
    except:
        pass
    
    iv = os.urandom(16)   #generate an iv

    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message) + padder.finalize()
    message = padded_data
        
    #calling the default AES CBC mode
    cipher = Cipher(algorithms.AES(EncKey), modes.CBC(iv), backend=default_backend())
    #creating an encryptor object
    encryptor = cipher.encryptor()
    #generating cipher text
    ct = encryptor.update(message) + encryptor.finalize()
    
    #step 2: HMAC
    # create a tag
    tag = hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
    tag.update(ct)
    tag = tag.finalize()
    print(type(tag))
#     print(type(tag.update(ct)))
#     print(type(tag.finalize()))
        
    return(ct, iv, tag) 


# MACVerification-then-Decrypt
def MydecryptMAC(ct, iv, tag, EncKey, HMACKey):
    print(tag)
    print(ct)

    #Step 1: Verify
    h= hmac.HMAC(HMACKey, hashes.SHA256(), backend=default_backend())
    h.update(ct)
    h = h.finalize()
    
    if h == tag:
        print("yahh:         verified, proceed to decrypt")

    else:
        print("noo: this is a hacker")

    
#     cipher = Cipher(algorithms.AES(EncKey), modes.CBC(iv), backend=default_backend())
#     #creating a decryptor object
#     decryptor = cipher.decryptor()
#     pt = decryptor.update(ct) + decryptor.finalize()
    
#     try:
#         unpadder = padding.PKCS7(128).unpadder()
#         pt = unpadder.update(pt) + unpadder.finalize()
#         return pt
#     except:
#         return pt



# execution code    
EncKey = os.urandom(32)
HMACKey = os.urandom(32)

m = "a secret message"
C, IV, tag = MyencryptMAC(m, EncKey, HMACKey)
# print("Encrypted Message with a tag:\n")
# print(result)
# print("\nDecrypted Message:")
# MydecryptMAC(C, IV, tag, EncKey, HMACKey).decode('utf8')
MydecryptMAC(C, IV, tag, EncKey, HMACKey)

<class 'bytes'>
b'\x97\xa6;e\xd1\xc46O\xfaM\xbe\xa4\x9b,~\x88o\xdfD1q\x06\xc0Y\x92\x83\xec\x93\xb3\xd8+\x07'
b'\x94~\xc7\xda\x00\x0f\xa1Lxk{\xbe\xca\x10]q\xd6\xe2\xc4\xbe\xb1\xf2vb3\x1e\x1cD\xe1GB\xb4'
yahh
